In [83]:
import thrift
import sys
import pprint
if sys.version_info[0] > 2:
    from urllib.parse import urlparse
else:
    from urlparse import urlparse
from thrift.transport import TTransport, TSocket, TSSLSocket, THttpClient
from thrift.protocol.TBinaryProtocol import TBinaryProtocol

from rpc_client.funicorn import FunicornService
from rpc_client.funicorn.ttypes import *
import numpy as np
import cv2
import sys

In [2]:
pp = pprint.PrettyPrinter(indent=2)
host = '0.0.0.0'
port = 8001
framed = True

socket = TSocket.TSocket(host, port)
transport = TTransport.TFramedTransport(socket)
protocol = TBinaryProtocol(transport)
client = FunicornService.Client(protocol)

In [3]:
def predict_img_bytes(img_bytes):
    transport.open()
    ret = None
    try:
        ret = client.predict_img_bytes(img_bytes)
    except Exception as e:
        print(e)
    finally:
        transport.close()
    return ret

def ping():
    transport.open()
    try:
        client.ping()
    except Exception as e:
        print(e)
    finally:
        transport.close()

In [140]:
ping()

In [141]:
# img_arr = cv2.imread('/Users/congvo/Downloads/0-5a7sHMpoIqMB4QYo.png')
img_arr = np.ones(shape=(640, 480, 3), dtype=np.uint8)
img_arr = cv2.resize(img_arr, (640, 480))
ret, img_flatten = cv2.imencode('.jpg', img_arr)
img_bytes = img_flatten.tobytes()
sys.getsizeof(img_bytes)/(1024*1024)

0.005207061767578125

In [142]:
img_flatten = np.frombuffer(img_bytes, dtype=np.uint8)

In [143]:
img_flatten

array([255, 216, 255, ...,   3, 255, 217], dtype=uint8)

In [144]:
img_arr_decoded = cv2.imdecode(img_flatten, cv2.IMREAD_ANYCOLOR)

In [145]:
img_arr_decoded.shape

(480, 640, 3)

In [146]:
%timeit predict_img_bytes(img_bytes)

21.7 ms ± 543 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
len(img_bytes)

143042

In [131]:
%timeit ret, img_flatten = cv2.imencode('.jpg', img_arr)

4.39 ms ± 508 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [132]:
# ~20ms